# Day 3 - Conversational AI - aka Chatbot!

In [9]:
# imports

import gradio as gr
import ollama

In [ ]:
MODEL = "gemma3:270m"
# Again, I'll be in scientist-mode and change this global during the lab

system_message = "You are a helpful assistant"

## And now, writing a new callback

We now need to write a function called:

`chat(message, history)`

Which will be a callback function we will give gradio.

### The job of this function

Take a message, take the prior conversation, and return the response.


In [11]:
def chat(message, history):
    return "bananas"

In [12]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [13]:
import json
def chat(message, history):
    with open("history.json", "w") as f:
        json.dump(history, f, indent=2)

    return f"You said {message} and the history is {history} but I still say bananas"

In [14]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## OK! Let's write a slightly better chat callback!

In [15]:
def chat(message, history):
    print(message)
    
    #  Extract text safely (new Gradio format)
    if isinstance(message, dict):
        message_text = message["content"][0]["text"]
    elif isinstance(message, list):
        message_text = message[0]["text"]
    else:
        message_text = message

    # Convert history to Ollama format
    formatted_history = []

    for msg in history:
        if isinstance(msg["content"], list):
            text = msg["content"][0]["text"]
        else:
            text = msg["content"]

        formatted_history.append({
            "role": msg["role"],
            "content": text
        })

    messages = (
        [{"role": "system", "content": system_message}]
        + formatted_history
        + [{"role": "user", "content": message_text}]
    )

    response = ollama.chat(model=MODEL, messages=messages)
    
    #  Return in Gradio expected format
    return {
        "role": "assistant",
        "content": response["message"]["content"]
    }




#  The Code

```python
if isinstance(message, dict):
    message_text = message["content"][0]["text"]
elif isinstance(message, list):
    message_text = message[0]["text"]
else:
    message_text = message
```

---

#  First Understand the Problem

In newer versions of **Gradio**, the message can be:

### Case 1 — Dict format

```python
{
   "role": "user",
   "content": [
       {"type": "text", "text": "hello"}
   ]
}
```

### Case 2 — List format

```python
[
   {"type": "text", "text": "hello"}
]
```

### Case 3 — Plain string

```python
"hello"
```

Your model (Ollama, OpenAI, etc.) expects:

```python
"hello"
```

So we must extract the text safely.

---

# Line-by-Line Deep Explanation

---

## 🔹 1️⃣ `isinstance(message, dict)`

### What is `isinstance()`?

Python built-in function:

```python
isinstance(object, datatype)
```

Returns:

* `True` → if object matches datatype
* `False` → otherwise

Example:

```python
isinstance("hello", str)  # True
isinstance(5, int)        # True
isinstance([], list)      # True
```

---

### Here:

```python
if isinstance(message, dict):
```

We check:

👉 Is message a dictionary?

---

### Example Input

```python
message = {
   "role": "user",
   "content": [
       {"type": "text", "text": "Explain AI"}
   ]
}
```

This is a `dict`.

So this block runs:

```python
message_text = message["content"][0]["text"]
```

---

### Step-by-Step Breakdown

```python
message["content"]
```

Returns:

```python
[
   {"type": "text", "text": "Explain AI"}
]
```

Then:

```python
message["content"][0]
```

Returns:

```python
{"type": "text", "text": "Explain AI"}
```

Then:

```python
["text"]
```

Returns:

```python
"Explain AI"
```

So final:

```python
message_text = "Explain AI"
```

---

# 🔹 2️⃣ `elif isinstance(message, list)`

Now suppose message is:

```python
message = [
   {"type": "text", "text": "Hello"}
]
```

This is a list.

So second condition runs:

```python
message_text = message[0]["text"]
```

Breakdown:

```python
message[0]
```

Returns:

```python
{"type": "text", "text": "Hello"}
```

Then:

```python
["text"]
```

Returns:

```python
"Hello"
```

---

# 🔹 3️⃣ Else Condition

```python
else:
    message_text = message
```

This handles:

```python
message = "Hello"
```

Since it's already string:

We don't need to extract anything.

---

#  Why This Code Is Important

Without this safety check, you might do:

```python
message_text = message["content"][0]["text"]
```

But if message is:

```python
"hello"
```

Python throws:

```
TypeError: string indices must be integers
```

Or if message is list:

```
TypeError: list indices must be integers
```

This code prevents runtime crashes.

---

#  Real Execution Flow Example

Suppose Gradio sends:

```python
message = [
   {"type": "text", "text": "What is Python?"}
]
```

Execution:

```
isinstance(dict)? → False
isinstance(list)? → True
```

So:

```python
message_text = "What is Python?"
```

Now safe to send to model.

---

#  Why Multiple Formats Exist?

Because new Gradio supports:

* Text
* Images
* Audio
* Files
* Multimodal messages

So content is stored as:

```python
[
   {"type": "text", "text": "..."},
   {"type": "image", "url": "..."}
]
```

Your code extracts only text part.


---

#  Concept Behind This Pattern

This is called:

> Defensive Programming

We handle all possible input types safely.

Very important in production systems.

---

#  Interview-Level Explanation (Short Version)

If interviewer asks:

Why use `isinstance()` here?

Answer:

1. Gradio may send message in dict, list, or string format.
2. LLM APIs expect plain string content.
3. This ensures safe extraction and prevents type errors.

---

#  Final Mental Model

Think like this:

```
Gradio gives → Complex object
LLM needs   → Simple string
This block  → Converts safely
```

---



This block is responsible for:

> Converting Gradio’s chat history into clean LLM-compatible format.

Let’s break it down **step-by-step with real examples**.

---

#  The Code

```python
formatted_history = []

for msg in history:
    if isinstance(msg["content"], list):
        text = msg["content"][0]["text"]
    else:
        text = msg["content"]

    formatted_history.append({
        "role": msg["role"],
        "content": text
    })
```

---

#  First Understand: What Is `history`?

In **Gradio ChatInterface**,
`history` is a list of previous messages.

Example:

```python
history = [
    {
        "role": "user",
        "content": [{"type": "text", "text": "Hi"}]
    },
    {
        "role": "assistant",
        "content": [{"type": "text", "text": "Hello!"}]
    }
]
```

Notice:

👉 `content` is a **list** (because Gradio supports multimodal input)

But your LLM (Ollama/OpenAI/etc.) expects:

```python
[
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hello!"}
]
```

So we must clean it.

---

#  Line-by-Line Explanation

---

## 🔹 1️⃣ Create Empty List

```python
formatted_history = []
```

We create a new list to store cleaned messages.

Why?

Because we don’t want to modify original history directly.

---

## 🔹 2️⃣ Loop Through Each Message

```python
for msg in history:
```

Each `msg` looks like:

```python
{
    "role": "user",
    "content": [{"type": "text", "text": "Hi"}]
}
```

First loop:

```
msg = {"role": "user", "content": [...]}
```

Second loop:

```
msg = {"role": "assistant", "content": [...]}
```

---

## 🔹 3️⃣ Check Content Type

```python
if isinstance(msg["content"], list):
```

Why check this?

Because Gradio content may be:

### Case A — List (multimodal format)

```python
[
   {"type": "text", "text": "Hi"}
]
```

### Case B — Already string

```python
"Hi"
```

We handle both safely.

---

## 🔹 4️⃣ Extract Text If It’s List

```python
text = msg["content"][0]["text"]
```

Step-by-step:

```python
msg["content"]
```

Returns:

```python
[
   {"type": "text", "text": "Hi"}
]
```

Then:

```python
[0]
```

Returns:

```python
{"type": "text", "text": "Hi"}
```

Then:

```python
["text"]
```

Returns:

```python
"Hi"
```

So now:

```python
text = "Hi"
```

---

## 🔹 5️⃣ Else Case

```python
else:
    text = msg["content"]
```

If content is already string:

```python
msg = {
    "role": "assistant",
    "content": "Hello!"
}
```

Then:

```python
text = "Hello!"
```

---

## 🔹 6️⃣ Append Clean Message

```python
formatted_history.append({
    "role": msg["role"],
    "content": text
})
```

This creates:

```python
{
   "role": "user",
   "content": "Hi"
}
```

And adds it to new list.

---

#  Final Output Example

### Original `history`

```python
[
    {
        "role": "user",
        "content": [{"type": "text", "text": "Hi"}]
    },
    {
        "role": "assistant",
        "content": [{"type": "text", "text": "Hello!"}]
    }
]
```

### After Processing

```python
formatted_history = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hello!"}
]
```

Now this is compatible with:

* Ollama
* OpenAI
* Mistral
* DeepSeek
* Most modern LLM APIs

---

#  Why This Is Very Important

If you don’t clean it:

Your API call receives:

```python
"content": [{"type": "text", "text": "Hi"}]
```

But LLM expects:

```python
"content": "Hi"
```

Then you get:

```
ValidationError: content must be string
```

Which you were seeing earlier.

---

#  Concept: Data Normalization

This block performs:

> Input Normalization

We convert UI-specific structure → Model-compatible structure.

This is very common in production systems.

---

#  Architecture View

```
Gradio History (complex)
        ↓
Normalize content
        ↓
formatted_history (clean)
        ↓
Send to model
```

---

#  Interview-Level Explanation (Short Version)

Why do we loop through history?

1. Gradio stores content as list (multimodal).
2. LLM APIs expect plain string.
3. So we extract text safely before sending to model.


---

#  Final Mental Model

Think of this like:

```
history = complex UI format
formatted_history = clean API format
```




#  The Code

```python
messages = (
    [{"role": "system", "content": system_message}]
    + formatted_history
    + [{"role": "user", "content": message_text}]
)

response = ollama.chat(model=MODEL, messages=messages)

return {
    "role": "assistant",
    "content": response["message"]["content"]
}
```

---

#  Step 1: Creating `messages`

## 🔹 What Is `messages`?

It is a **list of dictionaries**.

Modern LLM APIs expect input in this format:

```python
[
  {"role": "system", "content": "..."},
  {"role": "user", "content": "..."},
  {"role": "assistant", "content": "..."}
]
```

This is the standard conversational structure used by:

* Ollama
* OpenAI API
* Mistral API
* DeepSeek API
* Almost all chat-based LLMs

---

# 🔹 Breaking Down the List Concatenation

```python
[{"role": "system", "content": system_message}]
+ formatted_history
+ [{"role": "user", "content": message_text}]
```

This is **list concatenation** in Python.

Example:

```python
a = [1]
b = [2,3]
c = [4]

result = a + b + c
```

Output:

```python
[1,2,3,4]
```

Same concept here.

---

#  Step 2: What Each Part Means

---

## 🔹 1️⃣ System Message

```python
{"role": "system", "content": system_message}
```

This controls AI behavior.

Example:

```python
system_message = "You are a helpful coding assistant."
```

This influences:

* Tone
* Personality
* Response style
* Rules

💡 The model reads system message first.

---

## 🔹 2️⃣ `formatted_history`

Example:

```python
[
  {"role": "user", "content": "Hi"},
  {"role": "assistant", "content": "Hello!"}
]
```

This gives conversation memory.

Without this → model forgets previous messages.

---

## 🔹 3️⃣ Current User Message

```python
{"role": "user", "content": message_text}
```

This is the new question.

---

# Final `messages` Example

Suppose:

```
system_message = "You are helpful."
message_text = "Explain AI."
```

Then:

```python
messages = [
  {"role": "system", "content": "You are helpful."},
  {"role": "user", "content": "Hi"},
  {"role": "assistant", "content": "Hello!"},
  {"role": "user", "content": "Explain AI."}
]
```

This is exactly what the LLM sees.

---

#  Step 3: Calling Ollama

```python
response = ollama.chat(model=MODEL, messages=messages)
```

This sends:

* Model name (e.g., `"phi3"`)
* Entire conversation list

To Ollama runtime.

Ollama processes it and returns:

```python
{
   "model": "phi3",
   "message": {
       "role": "assistant",
       "content": "AI stands for Artificial Intelligence..."
   }
}
```

---

#  Step 4: Extract Model Reply

```python
response["message"]["content"]
```

Breakdown:

```python
response["message"]
```

Returns:

```python
{
   "role": "assistant",
   "content": "AI stands for Artificial Intelligence..."
}
```

Then:

```python
["content"]
```

Returns:

```python
"AI stands for Artificial Intelligence..."
```

---

# 🔹 Step 5: Return in Gradio Format

```python
return {
    "role": "assistant",
    "content": response["message"]["content"]
}
```

Why return dictionary?

Because new **Gradio** expects:

```python
{
  "role": "assistant",
  "content": "text"
}
```

If you return just string:

```
ValidationError
```

---

# 🧠 Architecture Flow

```
User types message
        ↓
Gradio sends message + history
        ↓
Build messages list
        ↓
Send to Ollama
        ↓
Get response
        ↓
Return in Gradio format
        ↓
UI displays assistant reply
```

---

#  Why This Pattern Is Important

This is called:

> Role-Based Chat Formatting

It allows the model to understand:

* Who is speaking
* What is instruction
* What is past memory
* What is current question

Without roles, conversation quality drops.

---

#  Interview-Level Summary (3 Points)

If interviewer asks:

How does chat API work internally?

You say:

1. We construct a list of role-based messages (system, user, assistant).
2. We append history + new input to maintain context.
3. We send entire conversation to model, which generates next assistant response.

---

# Final Mental Model

Think of `messages` as:

```
Full conversation transcript
```

And the model generates:

```
Next assistant reply
```



In [16]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


## OK let's keep going!

Using a system message to add context, and to give an example answer.. this is "one shot prompting" again

In [17]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [18]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [19]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
import gradio as gr
import json
MODEL = "gemma3:270m"
def chat(message, history):
    print(message)
    print(history)
    print(type(history))
    # #  Extract text safely (new Gradio format)
    # if isinstance(message, dict):
    #     message_text = message["content"][0]["text"]
    # elif isinstance(message, list):
    #     message_text = message[0]["text"]
    # else:
    message_text = message

    # Convert history to Ollama format
    formatted_history = []
    a = [{"role" : msg["role"], "content" : msg["content"][0]["text"]} for msg in history]
    print("a ", a)

    for msg in history:
        if isinstance(msg["content"], list):
            text = msg["content"][0]["text"]
        else:
            text = msg["content"]

        formatted_history.append({
            "role": msg["role"],
            "content": text
        })
    print("hist ",formatted_history)
    messages = (
        [{"role": "system", "content": system_message}]
        + formatted_history
        + [{"role": "user", "content": message_text}]
    )
    print("msg ",messages)

    response = ollama.chat(model=MODEL, messages=messages)
    print("res ",response)
    with open("response.json", "w") as f:
        json.dump(response.model_dump(), f, indent=4)
    #  Return in Gradio expected format
    return {
        "role": "assistant",
        "content": response["message"]["content"]
    }




In [ ]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


res  model='phi3' created_at='2026-02-17T17:58:06.5311164Z' done=True done_reason='stop' total_duration=90518133300 load_duration=5252818400 prompt_eval_count=160 prompt_eval_duration=15555028000 eval_count=291 eval_duration=69186407900 message=Message(role='assistant', content="Hello there! Welcome to our fashion-forward store where style meets savings. We've got a special promotion going on right now: Hats have been slashed by an amazing 60% off, making them the steal of the season here in Paris. If you're eyeing some trendy headwear or just looking to top off your ensemble with something chic and affordable, why not take a peek at our hat collection? They are crafted by renowned designers from around the globe – styles ranging from classic fedoras for timeless looks to avant-garde beanies that make bold statements. Plus, we have exclusive Parisian designs you won't find anywhere else!\n\nIf hats aren’t your thing or if there is something specific on your mind like shoes, I assure yo

Traceback (most recent call last):
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\blocks.py", line 2152, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\blocks.py", line 1627, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dell\Documents\gen

hello
[]
<class 'list'>
a  []
hist  []
msg  [{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get.\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!"}, {'role': 'user', 'content': 'hello'}]
res  model='gemma3:270m' created_at='2026-02-17T17:58:12.6095641Z' done=True done_reason='stop' total_duration=1821276200 load_duration=199442200 prompt_eval_count=142 prompt_eval_duration=60382100 eval_count=84 eval_duration=1475249100 message=Message(role='assistant', content="Hi there! Thank you for your interest in our sal

Traceback (most recent call last):
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\blocks.py", line 2152, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\Dell\Documents\genai\day10\myenv\Lib\site-packages\gradio\blocks.py", line 1627, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dell\Documents\gen